In [ ]:
import requests
import pandas as pd

url = "https://covid-193.p.rapidapi.com/statistics"

headers = {
	"X-RapidAPI-Key": "d676a2844emsh62b86dd14d22f62p1be05fjsnae94af9aaeda",
	"X-RapidAPI-Host": "covid-193.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

data = response.json()

# Extract the statistics data
#Here, we're checking if there's a part in the decoded message called 'response' and if it's not empty. 
#If it's there and has some information, we take that information and call it statistics.
if 'response' in data and data['response']:
    statistics = data['response']
    
    # Create DataFrame
    df = pd.DataFrame(statistics)
    
    # Display DataFrame or perform further operations
    #print(df)
#else:
 #   print("No data found.")

    

In [ ]:
df

In [ ]:
#Data Cleaning

In [ ]:
#Extract only the time
df["Time"] = df["time"].str.split("T").str[1]
df["Time"]= df["Time"].str.split("+").str[0]
df.shape
df.info()#This would show some columns have null
df.head()

In [ ]:
#df['Time'] = pd.to_datetime(df['Time'])
df.info()

In [ ]:
#Changing time column to date and time, expanding them in two columns respectively
#df["Date"] = df.Time.dt.date
#df["Time"] = df.Time.dt.time

In [ ]:
#Drop the the time column cause it's no longer needed
df.drop(columns = ["time"],inplace = True)

In [ ]:
pd.DataFrame(df['cases'])
df['cases'].apply(pd.Series)
additinal_columns = ['New Cases', 'Active Cases', 'Critical Cases', 'Recovered Cases', 'Cases per 1M Population', 'Total Cases']
df[additinal_columns[0:6]] = df['cases'].apply(pd.Series)


In [ ]:
pd.DataFrame(df['deaths'])
df['deaths'].apply(pd.Series)
additinal_columns_d = ['New deaths',  'deaths per 1M Population', 'Total deaths']
df[additinal_columns_d[0:3]] = df['deaths'].apply(pd.Series)


In [ ]:
pd.DataFrame(df['tests'])
df['tests'].apply(pd.Series)
additinal_columns_d = ['tests per 1M Population', 'Total tests']
df[additinal_columns_d[0:3]] = df['tests'].apply(pd.Series)


In [ ]:
#Drop the the time column cause it's no longer needed
df.drop(columns = ["cases","deaths","tests"],inplace = True)

In [ ]:
# Replace NaN values with 0 so as not to lose some vital info
df.fillna(0, inplace=True)


In [ ]:
df['Cases per 1M Population'] = df['Cases per 1M Population'].astype(float)
df['deaths per 1M Population'] = df['deaths per 1M Population'].astype(float)
df['New deaths'] = df['New deaths'].astype(float)
df['tests per 1M Population'] = df['tests per 1M Population'].astype(float)

In [ ]:
df.info()
df

In [ ]:
#pip install sqlalchemy psycopg2

In [ ]:
import psycopg2

# Connect to your PostgreSQL database
conn = psycopg2.connect(
    dbname="--",
    user="--",
    password='--', 
    host='--', 
    port= '--'
)

# Create a cursor object to execute SQL commands
cur = conn.cursor()

# Define the SQL statement to create a table
create_table_query = '''
CREATE TABLE IF NOT EXISTS Covid19_Data (
    continent VARCHAR(50),
    country VARCHAR(50),
    population VARCHAR(50),
    Time TIME,
    Date DATE,
    NewCases INTEGER,
    ActiveCases INTEGER,
    CriticalCases INTEGER,
    RecoveredCases INTEGER,
    CasesperMPopulation INTEGER,
    TotalCases INTEGER,
    Newdeaths INTEGER,
    deathsperMPopulation INTEGER,
    Totaldeaths INTEGER,
    testsperMPopulation INTEGER,
    Totaltests INTEGER
);

'''

# Execute the SQL command to create the table
cur.execute(create_table_query)

# Commit the transaction
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

print("Table created successfully!")


In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Assuming you have a DataFrame named df with your data

# Define your PostgreSQL connection parameters
username = '--'
password = '--'
host = '--'
port = '--'
database = 'PersonalProjects_DB'

# Create a SQLAlchemy engine to connect to your PostgreSQL database
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')

# Push the data from the DataFrame into the PostgreSQL table
df.to_sql('Covid19_Data', con=engine, if_exists='append',index=False)
